In [9]:
! pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
time: 2.24 s


In [10]:
%load_ext autotime
%matplotlib inline

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.69 ms


In [11]:
import os
from pyspark.sql import SparkSession
import pymystem3
from pyspark.sql.types import BooleanType, IntegerType, StringType, FloatType, ArrayType
from pyspark.sql.functions import dayofyear, weekofyear, to_date, concat_ws, collect_list, year, udf, col, size, countDistinct, lit, sum, expr 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
import re
from pyspark.ml.feature import NGram

import multiprocessing as mp
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer

import artm
import glob
import os
import pandas as pd
from pyarrow import parquet
import pickle
import tqdm

from pymystem3 import Mystem

time: 1.48 s


In [12]:
full_datasets_path = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/'

time: 308 µs


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
dataset_names = ["20newsgroups",
                 "amazon_food",
                 "hotel-reviews",
                 "lenta_ru",
                 "ads_data"
                ]

time: 330 µs


In [6]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/

20newsgroups	     NIPS	  hotel-reviews  synthetic
20newsgroups_sample  amazon_food  lenta_ru	 test_set_data
time: 402 ms


In [36]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/

20newsgroups  ads_data	amazon_food  hotel-reviews  lenta_ru
time: 434 ms


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

time: 4.69 s


# Processing instruments

In [14]:
# dataset cleaning 
r_num = re.compile(r'([0-9]+)')
r_punct = re.compile(r'[."\[\]/,()!?;:*#|\\%^$&{}~_`=-@]')
r_white_space = re.compile(r'\s{2,}')
r_words = re.compile(r'\W+')
re1 = re.compile(r'  +')
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
r_rus = re.compile(r'[а-яА-Я]\w+')
r_html = re.compile(r'(\<[^>]*\>)')
r_email = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")

stop = stopwords.words("russian") + ['который', 'это', 'весьма', 'вполне', 'наверное']

m = Mystem()

def process_punkt(text):
    text = r_punct.sub(" ", text)
    text = r_num.sub(" ", text)
    text = r_white_space.sub(" ", text)
    return text.strip()


def new_html(text):
    text = r_html.sub("", text)
    return text


def lemmatize_text_ru(text):

    try:
        text = new_html(text)
    except:
        return ''

    text = text.lower()
    text = process_punkt(text)
    text = re.findall(r_rus, text)
    text = ' '.join(text)

    try:
        tokens = r_words.split(text)
    except:
        return ''

    tokens = (x for x in tokens if len(x) >= 2 and not x.isdigit())
    text = ' '.join(tokens)
    tokens = m.lemmatize(text)
    tokens = (x for x in tokens if x not in stop)
    tokens = (x for x in tokens if x.isalpha())
    text = ' '.join(tokens)

    return text

def lemmatize_text_en(line):
    try:
        line = new_html(line)
    except:
        return ''
    lmtzr = WordNetLemmatizer()    
    stop = stopwords.words('english')
    text_token = CountVectorizer().build_tokenizer()(line.lower())
    text_token = (x for x in text_token if len(x) > 2 and not x.isdigit())
    text_rmstop = (i for i in text_token if i not in stop)
    text_rmstop = (x for x in text_rmstop if x.isalpha())
    text_stem = ' '.join([lmtzr.lemmatize(w) for w in text_rmstop])
    #stext_stem = remove_more_html(text_stem)
    return text_stem
    

def text_to_tokens(text):
    return text.split()


def remove_more_html(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('<', ' ').replace('>', ' ').replace('#36;', '$').replace(
        '\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ').replace('img', ' ').replace('class', ' ').replace(
        'src', ' ').replace('alt', ' ').replace('email', ' ').replace('icq', ' ').replace(
        'href', ' ').replace('mem', ' ').replace('link', ' ').replace('mention', ' ').replace(
        'onclick', ' ').replace('icq', ' ').replace('onmouseover', ' ').replace(
        'local', ' ').replace('key', ' ').replace('target', ' ').replace('amp', ' ').replace(
        'section', ' ').replace('search', ' ').replace('css', ' ').replace('style', ' ').replace(
        'cc', ' ').replace('text', ' ').replace("img", ' ').replace("expand", ' ').replace(
        "text", ' ').replace('\n', ' ').replace('dnum', ' ').replace('xnum', ' ').replace(
        'nnum', ' ').replace("ubuntu", ' ').replace('server', ' ').replace('port', ' ').replace('nntp', ' ')
    return re1.sub(' ', html.unescape(x))


def clear_url(text):
    return re.sub(url_pattern, ' ', text)


def tokens_count(text):
    return len(text.split())


def get_rid_of_num(text):
    tokens = text.split()
    tokens = [i for i in tokens if i != 'num']
    return ' '.join(tokens)

/opt/conda/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Callable
Installing mystem to /home/jovyan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


time: 2.53 s


In [16]:
def clean_dataset(dataset_path, save_dataset_path, column_name, language):
    dataset = pd.read_csv(dataset_path)
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    p = mp.Pool(mp.cpu_count())
    if language == "ru":
        dataset['processed_text'] = p.map(lemmatize_text_ru, dataset[column_name])
    if language == 'en':
        dataset['processed_text'] = p.map(lemmatize_text_en, dataset[column_name])
    dataset['tokens_count'] = p.map(tokens_count, dataset['processed_text'])
    
    print('Initial dataset shape: ', dataset.shape)
    dataset = dataset[dataset['tokens_count'] >= 3]
    print('Filtered dataset shape: ', dataset.shape)
    
    dataset.to_csv(save_dataset_path, index=None)
    print('Dataset is saved')
    
    return
    

def get_words_dict(text, stop_list):
    
    all_words = text
    words = sorted(set(all_words)-stop_list)
    
    return {w: all_words.count(w) for w in words}

def return_string_part(name_type, text):
    
    tokens = text.split()
    tokens = [item for item in tokens if item != '']
    tokens_dict = get_words_dict(tokens, set())
    
    return " |" + name_type + ' ' + ' '.join(['{}:{}'.format(k, v) for k, v in tokens_dict.items()])

def prepare_voc(batches_dir, vw_path, data_path, column_name='processed_text'):
    
    print('Starting...')
    with open(vw_path, 'w', encoding='utf8') as ofile:
        num_parts = 0
        try:
            for file in os.listdir(data_path):
                if file.startswith('part'):
                    print('part_{}'.format(num_parts), end='\r')
                    if file.split('.')[-1] == 'csv':
                        part=pd.read_csv(os.path.join(data_path, file))
                    else:
                        part = pd.read_parquet(os.path.join(data_path, file))
                    part_processed = part[column_name].tolist()
                    for text in part_processed:
                        result = return_string_part('@default_class', text)
                        ofile.write(result + '\n')
                    num_parts += 1

        except NotADirectoryError:
            print('part 1/1')
            part = pd.read_csv(data_path)
            part_processed = part[column_name].tolist()
            for text in part_processed:
                result = return_string_part('@default_class', text)
                ofile.write(result + '\n')

    print(' batches {} \n vocabulary {} \n are ready'.format(batches_dir, vw_path))
    
def prepare_batch_vectorizer(batches_dir, vw_path, data_path, column_name='processed_text'):
#     if not glob.glob(os.path.join(batches_dir, "*")):
    prepare_voc(batches_dir, vw_path, data_path, column_name=column_name)
    batch_vectorizer = artm.BatchVectorizer(data_path=vw_path,
        data_format="vowpal_wabbit", 
        target_folder=batches_dir,
    batch_size=100)
#     else:
#         batch_vectorizer = artm.BatchVectorizer(data_path=batches_dir, data_format='batches')
        
    return batch_vectorizer

def vocab_preparation(VOCAB_PATH, DICTIONARY_PATH):
    
    if not os.path.exists(VOCAB_PATH):
        with open(DICTIONARY_PATH, 'r') as dictionary_file:
            with open(VOCAB_PATH, 'w') as vocab_file:
                dictionary_file.readline()
                dictionary_file.readline()
                for line in dictionary_file:
                    elems = re.split(', ', line)
                    vocab_file.write(' '.join(elems[:2]) + '\n')

                    
def prepearing_cooc_dict(BATCHES_DIR, WV_PATH, VOCAB_PATH, COOC_DICTIONARY_PATH, 
                         cooc_file_path_tf, cooc_file_path_df, ppmi_dict_tf, ppmi_dict_df):
    print(f'BATCHES_DIR: {BATCHES_DIR}')
    print(f'WV_PATH: {WV_PATH}')
    print(f'VOCAB_PATH: {VOCAB_PATH}')
    print(f'COOC_DICTIONARY_PATH: {COOC_DICTIONARY_PATH}')
    print(f'cooc_file_path_tf: {cooc_file_path_tf}')
    print(f'cooc_file_path_df: {cooc_file_path_df}')
    print(f'ppmi_dict_tf: {ppmi_dict_tf}')
    print(f'ppmi_dict_df: {ppmi_dict_df}')
    
    ! bigartm -v $VOCAB_PATH -c $WV_PATH --cooc-window 10 --write-cooc-tf $cooc_file_path_tf --write-cooc-df $cooc_file_path_df --write-ppmi-tf $ppmi_dict_tf --write-ppmi-df $ppmi_dict_df
    
    cooc_dict = artm.Dictionary()
    cooc_dict.gather(
        data_path=BATCHES_DIR, 
        cooc_file_path=ppmi_dict_tf,
        vocab_file_path=VOCAB_PATH,
        symmetric_cooc_values=True)
    cooc_dict.save_text(COOC_DICTIONARY_PATH)
                    
        
def mutual_info_dict_preparation(fname):

    tokens_dict = {}

    with open(fname) as handle:
        for ix, line in enumerate(handle):
            list_of_words = line.strip().split()
            word_1 = list_of_words[0]
            for word_val in list_of_words[1:]:
                word_2, value = word_val.split(':')
                tokens_dict['{}_{}'.format(word_1, word_2)] = float(value)
                tokens_dict['{}_{}'.format(word_2, word_1)] = float(value)
    return tokens_dict
        
def prepare_all(DATASET_PATH, column_name, dataset_lang):
    
    SAVE_DATASET_PATH = os.path.join(DATASET_PATH, 'processed_dataset.csv')
    BATCHES_DIR = os.path.join(DATASET_PATH, 'batches')
    WV_PATH = os.path.join(DATASET_PATH, 'wv.txt')
    DOCUMENTS_TO_BATCH_PATH = os.path.join(DATASET_PATH, 'dataset.csv')
    DICTIONARY_PATH = os.path.join(DATASET_PATH, 'dictionary.txt')
    VOCAB_PATH = os.path.join(DATASET_PATH, 'vocab.txt')
    COOC_DICTIONARY_PATH = os.path.join(DATASET_PATH, 'cooc_dictionary.txt')
    cooc_file_path_tf = os.path.join(DATASET_PATH, 'cooc_tf.txt')
    cooc_file_path_df = os.path.join(DATASET_PATH, 'cooc_df.txt')
    ppmi_dict_tf = os.path.join(DATASET_PATH,'ppmi_tf.txt')
    ppmi_dict_df = os.path.join(DATASET_PATH, 'ppmi_df.txt')
    MUTUAL_INFO_DICT_PATH = os.path.join(DATASET_PATH,'mutual_info_dict.pkl')
    
    clean_dataset(DOCUMENTS_TO_BATCH_PATH, SAVE_DATASET_PATH, column_name, dataset_lang)
    
    batch_vectorizer = prepare_batch_vectorizer(BATCHES_DIR, WV_PATH, SAVE_DATASET_PATH)
    print('Batches are ready')
    
    my_dictionary = artm.Dictionary()
    my_dictionary.gather(data_path=BATCHES_DIR, vocab_file_path=WV_PATH)
    my_dictionary.filter(min_df=3, max_df=0.95, class_id='text')
    my_dictionary.save_text(DICTIONARY_PATH)
    print('Dictionary is ready')
    
    vocab_preparation(VOCAB_PATH, DICTIONARY_PATH)
    prepearing_cooc_dict(BATCHES_DIR, WV_PATH, VOCAB_PATH, 
                         COOC_DICTIONARY_PATH, cooc_file_path_tf, 
                         cooc_file_path_df, ppmi_dict_tf, 
                         ppmi_dict_df)
    
    mutual_info_dict = mutual_info_dict_preparation(ppmi_dict_tf)
    with open(MUTUAL_INFO_DICT_PATH, 'wb') as handle:
        pickle.dump(mutual_info_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

time: 4.25 ms


# 20 Newsgroups full

In [18]:
NG_DATASET = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/'

time: 313 µs


In [19]:
! ls $NG_DATASET

batches		     dataset.csv	   ppmi_tf.txt
cooc_df.txt	     dictionary.txt	   processed_dataset.csv
cooc_dictionary.txt  mutual_info_dict.pkl  vocab.txt
cooc_tf.txt	     ppmi_df.txt	   wv.txt
time: 571 ms


In [69]:
ng_column_name = 'processed_text'
dataset_lang = 'en'

time: 428 µs


In [70]:
prepare_all(NG_DATASET, ng_column_name, dataset_lang)

Initial dataset shape:  (11299, 5)
Filtered dataset shape:  (11280, 5)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_df.txt
p

# Amazon Food

In [1]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food/

Reviews.csv		      cooc_dictionary.txt   ppmi_df.txt
Reviews_10000_lemmatized.csv  cooc_tf.txt	    ppmi_tf.txt
batches			      dictionary.txt	    test_set_data_voc.txt
cooc_df.txt		      mutual_info_dict.pkl  vocab.txt


In [71]:
amazon_food = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food/Reviews.csv')

time: 3.78 s


In [37]:
! cp /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food/Reviews.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


time: 962 ms


In [39]:
! mv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/Reviews.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/dataset.csv

time: 419 ms


In [11]:
amazon_food.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


time: 10 ms


In [34]:
amazon_food.shape

(568454, 10)

time: 4.1 ms


In [72]:
AMAZON_DATASET = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/'
text_column_name = 'Text'
dataset_lang = 'en'

prepare_all(AMAZON_DATASET, text_column_name, dataset_lang)

Initial dataset shape:  (568454, 12)
Filtered dataset shape:  (568423, 12)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/amazon_food/cooc_df.txt
ppmi_

In [59]:
! ls $AMAZON_DATASET

batches		     dataset.csv	   ppmi_tf.txt
cooc_df.txt	     dictionary.txt	   processed_dataset.csv
cooc_dictionary.txt  mutual_info_dict.pkl  vocab.txt
cooc_tf.txt	     ppmi_df.txt	   wv.txt
time: 428 ms


# Hotel reviews

In [45]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/hotel-reviews/Datafiniti_Hotel_Reviews.csv

/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/hotel-reviews/Datafiniti_Hotel_Reviews.csv
time: 431 ms


In [54]:
! cp /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/hotel-reviews/Datafiniti_Hotel_Reviews.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/

time: 905 ms


In [56]:
! mv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/Datafiniti_Hotel_Reviews.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/dataset.csv 

time: 446 ms


In [57]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews

dataset.csv
time: 443 ms


In [47]:
hotel_reviews = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/hotel-reviews/Datafiniti_Hotel_Reviews.csv')

time: 354 ms


In [87]:
hr_column_name = 'reviews.text'
HR_DATASET = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews'
dataset_lang = 'en'

prepare_all(HR_DATASET, hr_column_name, dataset_lang)

Initial dataset shape:  (10000, 27)
Filtered dataset shape:  (9877, 27)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/hotel-reviews/cooc

# Lenta.ru

In [74]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/lenta_ru

batches					  lenta-ru-news_sample_15000.csv
cooc_df.txt				  mutual_info_dict.pkl
cooc_dictionary.txt			  ppmi_df.txt
cooc_tf.txt				  ppmi_tf.txt
dictionary.txt				  test_set_data_voc.txt
lenta-ru-news.csv			  vocab.txt
lenta-ru-news_sample_10000_processed.csv
time: 540 ms


In [77]:
! cp /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/lenta_ru/lenta-ru-news.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/

time: 4.28 s


In [79]:
! mv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/lenta-ru-news.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/dataset.csv

time: 517 ms


In [80]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru

dataset.csv
time: 503 ms


In [63]:
lenta_data = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/lenta_ru/lenta-ru-news.csv')

time: 26.5 s


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
text_column = 'text'
lenta_dataset = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru'
dataset_lang = 'ru'

prepare_all(lenta_dataset, text_column, dataset_lang)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Initial dataset shape:  (800975, 8)
Filtered dataset shape:  (799602, 8)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/lenta_ru/cooc_df.txt
ppmi_dict_tf: /mnt/ess_storage/

# Ads dataset

In [90]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/test_set_data

banners_content_gz	cooc_dictionary.txt   ppmi_df.txt
banners_content_pd.csv	cooc_tf.txt	      ppmi_tf.txt
batches			dictionary.txt	      test_set_data_voc.txt
cooc_df.txt		mutual_info_dict.pkl  vocab.txt
time: 771 ms


In [91]:
banners = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/test_set_data/banners_content_pd.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


time: 1.56 s


In [92]:
banners.head()

,url,text,title,processed_text,tokens_count
0,http://autonew16.ru/catalog/tuning-aksessuari-...,Отзывы\nДоставка и оплата\nКонтакты\nКак заказ...,Купить тюнинг на ВАЗ (Лада) 21099 — аксессуары...,отзыв доставка оплата контакт заказывать тюнин...,688
1,http://baza-creditov.ru/,Сумма кредита\nот\nдо\n7000\n200000 руб.\nПроц...,База кредитов: займы и рассрочки,сумма кредит руб процентный ставка срок кредит...,424
2,http://dom.tv.of.by/,НОЖ кредитка в подарок\nневозможно поцарапать\...,Часы бизнес-класса DOM - невозможно поцарапать...,нож кредитка подарок невозможно поцарапать час...,229
3,http://e96.ru/kids/baby_cribs/detskie_krovatki...,Товары для кухни\nВстраиваемая техника\nТовары...,Детские кроватки для новорожденных — купить не...,товар кухня встраивать техника товар дом аудио...,295
4,http://ekotopas.ru/servisnoe-obsluzhivanie/kon...,Разовое обслуживание ТОПАС\nКонсервация ТОПАС\...,Консервация септика Топас на зиму 🚿 ekoTopas,разовый обслуживание топас консервация топас р...,724


time: 24.2 ms


In [93]:
! cp /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/test_set_data/banners_content_pd.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/

time: 994 ms


In [94]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/

banners_content_pd.csv
time: 697 ms


In [95]:
! mv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/banners_content_pd.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/dataset.csv

time: 724 ms


In [96]:
text_column = 'processed_text'
ads_dataset = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data'
dataset_lang = 'ru'

prepare_all(ads_dataset, text_column, dataset_lang)

Initial dataset shape:  (10000, 5)
Filtered dataset shape:  (10000, 5)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/ads_data/cooc_df.txt
ppmi_dict_tf: /mnt/ess_storage/DN

# Testing

In [ ]:
TEST_FOLDER = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/test_folder'
TEST_DATASET_PATH = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups_sample'

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups_sample

In [ ]:
TEST_WV_PATH = os.path.join(TEST_DATASET_PATH, 'test_set_data_voc.txt')
TEST_VOCAB_PATH = os.path.join(TEST_DATASET_PATH, 'vocab.txt')
test_cooc_file_path_tf = os.path.join(TEST_FOLDER, 'cooc_tf.txt')
test_cooc_file_path_df = os.path.join(TEST_FOLDER, 'cooc_df.txt')
test_ppmi_dict_tf = os.path.join(TEST_FOLDER, 'ppmi_tf.txt')
test_ppmi_dict_df = os.path.join(TEST_FOLDER, 'ppmi_df.txt')

In [ ]:
! head -5 $TEST_WV_PATH

In [ ]:
! bigartm -c $TEST_WV_PATH -v $TEST_VOCAB_PATH --cooc-window 10 --write-cooc-tf $test_cooc_file_path_tf --write-cooc-df $test_cooc_file_path_df --write-ppmi-tf $test_ppmi_dict_tf --write-ppmi-df $test_ppmi_dict_df 

In [ ]:
! ls $TEST_FOLDER

In [ ]:
! bigartm

In [ ]:
! head -5 /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/vocab.txt

In [ ]:
! head -5 /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt

In [ ]:
! rm -r bigartm.WARNING

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups

In [ ]:
amazon_food = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food/Reviews.csv')

In [ ]:
amazon_food

In [ ]:
! ls /mnt/ess_storage/DN_1/archive/jupyter-notebooks-133/mailru/kh-mruml/notebooks/tm/Quality_Assessment

In [ ]:
! bigartm

# Dialogues datasets preparation

In [5]:
MAIN_DIALOGUE_DATA_PATH = "/mnt/ess_storage/DN_1/storage/qa-system-research/ClickHouseChat/datasets"

In [29]:
clickhouse_data = os.path.join(MAIN_DIALOGUE_DATA_PATH, "clickhouse_ru.combined_messages_by_author_and_time.json")
python_data = os.path.join(MAIN_DIALOGUE_DATA_PATH, "ru_python.combined_messages_by_author_and_time.json")
kino_data = os.path.join(MAIN_DIALOGUE_DATA_PATH, "kinota1k.combined_messages_by_author_and_time.json")

time: 474 µs


In [41]:
pd.read_json(python_data, lines=True)

,id,date,reply_to_msg_id,from_id,message,is_joint
0,1083186,2019-12-12 17:02:31,None,541532850.0,"А, ясно, просто в venv его не установил. Спасибо",0
1,1083185,2019-12-12 17:00:38,None,460960819.0,Привет. Есть кто подключал прокси с авторизаци...,0
2,1083184,2019-12-12 16:58:32,1083183,164184279.0,"В том, что нет модуля",0
3,1083183,2019-12-12 16:57:27,None,541532850.0,'''\nimport requests\nimport pprint\n\ndef get...,0
4,1083182,2019-12-12 16:56:30,None,771096498.0,"Добро пожаловать, @lolervt!\n\nНе забудь ознак...",0
...,...,...,...,...,...,...
852493,7,2015-12-18 14:00:20,None,462722.0,новый линк\nhttps://telegram.me/joinchat/AAcPg...,0
852494,6,2015-12-18 14:00:00,None,62240365.0,а чего мы так ждали 200 если могли вручную это...,0
852495,5,2015-12-18 13:59:44,None,63210888.0,"теперь игра ""найди приглашательный линк"" ;)",0
852496,4,2015-12-18 13:59:16,None,63210888.0,Ну теперь заживём :),0


time: 5.3 s


In [25]:
clickhouse_full_folder = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/'
python_full_folder = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/'
kino_full_folder = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/'

time: 612 µs


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
! mkdir $clickhouse_full_folder
! mkdir $python_full_folder
! mkdir $kino_full_folder

mkdir: cannot create directory ‘/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/’: File exists
mkdir: cannot create directory ‘/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/’: File exists
time: 809 ms


In [43]:
clickhouse_df = pd.read_json(clickhouse_data, lines=True)
python_df = pd.read_json(python_data, lines=True)
kino_df = pd.read_json(kino_data, lines=True)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


time: 6.37 s


In [44]:
clickhouse_df.to_csv(os.path.join(clickhouse_full_folder, "dataset.csv"), index=None)
python_df.to_csv(os.path.join(python_full_folder, "dataset.csv"), index=None)
kino_df.to_csv(os.path.join(kino_full_folder, "dataset.csv"), index=None)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


time: 5.24 s


In [ ]:
# clickhouse chat

In [39]:
text_column = 'message'
main_path = clickhouse_full_folder
dataset_lang = 'ru'

prepare_all(main_path, text_column, dataset_lang)

Initial dataset shape:  (139852, 8)
Filtered dataset shape:  (92677, 8)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clic

In [40]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/clickhouse_data

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


batches		     dataset.csv	   ppmi_tf.txt
cooc_df.txt	     dictionary.txt	   processed_dataset.csv
cooc_dictionary.txt  mutual_info_dict.pkl  vocab.txt
cooc_tf.txt	     ppmi_df.txt	   wv.txt
time: 420 ms


In [ ]:
# python chat

In [45]:
text_column = 'message'
main_path = python_full_folder
dataset_lang = 'ru'

prepare_all(main_path, text_column, dataset_lang)

Initial dataset shape:  (852498, 8)
Filtered dataset shape:  (456217, 8)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/python_data/cooc_df.txt
ppmi_di

In [ ]:
# kino chat

In [46]:
text_column = 'message'
main_path = kino_full_folder
dataset_lang = 'ru'

prepare_all(main_path, text_column, dataset_lang)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Initial dataset shape:  (62465, 8)
Filtered dataset shape:  (36631, 8)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/kino_data/cooc_df.txt
ppmi_dict_tf: /mnt/ess_st